# SNCF - OPEN DATA - API TRANSILIEN - "PROCHAINS DEPARTS"

In [ ]:
import os
import json
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [ ]:
spark = SparkSession.builder.appName("MS-SIO-HADOOP-PROJECT-KAFKA-CONSUMER").getOrCreate()

In [ ]:
df = spark \
   .readStream \
   .format("kafka") \
   .option("kafka.bootstrap.servers", "sandbox-hdp.hortonworks.com:6667") \
   .option("subscribe", "transilien-01") \
   .option("startingOffsets", "latest") \
   .load()

In [ ]:
df.printSchema()

schema: {'station':INT, 'timestamp':TIMESTAMP, 'train':INT, 'mission':STRING}  

In [ ]:
schema = StructType(
    [
        StructField("station", IntegerType(), True),
        StructField("latitude", FloatType(), True),
        StructField("longitude", FloatType(), True),
        StructField("timestamp", TimestampType(), True),
        StructField("train", IntegerType(), True),
        StructField("mission", StringType(), True)
    ]
)

In [ ]:
json_options = {"timestampFormat": "yyyy-MM-dd'T'HH:mm:ss.sss'Z'"}

In [ ]:
df = df.select(from_json(col("value").cast("string"), schema, json_options).alias("departure")).select("departure.*")

In [ ]:
df.printSchema()

In [ ]:
query = df \
      .writeStream \
      .outputMode("append") \
      .format("console") \
      .option('truncate', False) \
      .start()

In [ ]:
query.stop()

In [ ]:
spark.stop()